## Importing Libraries

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pickle
from sklearn.metrics import mean_absolute_error

## Uploading the Featured Data file

In [2]:
# match_format --> T20Is, ODIs
match_format = 'T20Is'
df = pd.read_csv('../Material/'+ match_format +'_Featured.csv')

In [3]:
df

,id,innings,batting_team,bowling_team,overs,over,ball,total_runs,player_dismissed,total,total_score,prev_30_runs,prev_30_wickets,total_wickets,prev_30_dot_balls,prev_30_boundaries,total_balls,run_rate,prev_30_run_rate,prev_30_strike_rate
0,211028,1,England,Australia,0.1,0,1,0,0,179,0,0,0,0,1,0,1,0.000000,0,0
1,211028,1,England,Australia,0.2,0,2,1,0,179,1,1,0,0,1,0,2,3.000000,3,50
2,211028,1,England,Australia,0.3,0,3,0,0,179,1,1,0,0,2,0,3,2.000000,2,33
3,211028,1,England,Australia,0.4,0,4,0,0,179,1,1,0,0,3,0,4,1.500000,1,25
4,211028,1,England,Australia,0.5,0,5,0,0,179,1,1,0,0,4,0,5,1.200000,1,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161909,1348651,2,New Zealand,India,11.3,11,3,0,0,66,66,31,3,8,13,2,69,5.739130,6,103
161910,1348651,2,New Zealand,India,11.4,11,4,0,0,66,66,30,3,8,14,2,70,5.657143,6,100
161911,1348651,2,New Zealand,India,11.5,11,5,0,1,66,66,28,4,9,15,2,71,5.577465,5,93
161912,1348651,2,New Zealand,India,11.6,11,6,0,0,66,66,27,4,9,16,2,72,5.500000,5,90


In [4]:
df = pd.get_dummies(data=df, columns=['batting_team', 'bowling_team'])

In [5]:
df.columns

Index(['id', 'innings', 'overs', 'over', 'ball', 'total_runs',
       'player_dismissed', 'total', 'total_score', 'prev_30_runs',
       'prev_30_wickets', 'total_wickets', 'prev_30_dot_balls',
       'prev_30_boundaries', 'total_balls', 'run_rate', 'prev_30_run_rate',
       'prev_30_strike_rate', 'batting_team_Australia',
       'batting_team_Bangladesh', 'batting_team_England', 'batting_team_India',
       'batting_team_New Zealand', 'batting_team_Pakistan',
       'batting_team_South Africa', 'batting_team_Sri Lanka',
       'batting_team_West Indies', 'bowling_team_Australia',
       'bowling_team_Bangladesh', 'bowling_team_England', 'bowling_team_India',
       'bowling_team_New Zealand', 'bowling_team_Pakistan',
       'bowling_team_South Africa', 'bowling_team_Sri Lanka',
       'bowling_team_West Indies'],
      dtype='object')

In [6]:
df = df[['id', 'batting_team_Australia',
       'batting_team_Bangladesh', 'batting_team_England', 'batting_team_India',
       'batting_team_New Zealand', 'batting_team_Pakistan',
       'batting_team_South Africa', 'batting_team_Sri Lanka',
       'batting_team_West Indies', 'bowling_team_Australia',
       'bowling_team_Bangladesh', 'bowling_team_England', 'bowling_team_India',
       'bowling_team_New Zealand', 'bowling_team_Pakistan',
       'bowling_team_South Africa', 'bowling_team_Sri Lanka',
       'bowling_team_West Indies',
        'overs', 'total_score', 'total_wickets', 'prev_30_runs',
       'prev_30_wickets', 'prev_30_dot_balls',
       'prev_30_boundaries', 'prev_30_run_rate',
       'prev_30_strike_rate', 'total']]

### train_test_split

In [7]:
X = df.drop(labels=['total', 'id'], axis=1)
y = df['total'].values

In [8]:
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y) 

In [9]:
X_train = X_train.values
X_test = X_test.values
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)

In [10]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(121435, 27) (40479, 27) (121435,) (40479,)


## Training LR Model

In [11]:
LR_model = LinearRegression()
LR_model.fit(X_train, y_train)

LinearRegression()

In [12]:
# Creating a pickle file for the classifier
filename = '../Material/lr-model.pkl'
pickle.dump(LR_model, open(filename, 'wb'))

In [13]:
prediction = LR_model.predict(X_test)
meanError = int(mean_absolute_error(y_test, prediction))
print(meanError)

18


In [14]:
# Function to predict the scores
def score_prediction(Bat_Team, Bowl_Team, overs, total_score, total_wickets,
                     prev_30_runs, prev_30_wickets, prev_30_dot_balls, prev_30_boundaries, prev_30_run_rate, prev_30_strike_rate):
    temp_array = list()
    
    if Bat_Team == 'AUS':
        temp_array += [1,0,0,0,0,0,0,0,0]
    elif Bat_Team == 'BAN':
        temp_array += [0,1,0,0,0,0,0,0,0]
    elif Bat_Team == 'ENG':
        temp_array += [0,0,1,0,0,0,0,0,0]
    elif Bat_Team == 'IND':
        temp_array += [0,0,0,1,0,0,0,0,0]
    elif Bat_Team == 'NZ':
        temp_array += [0,0,0,0,1,0,0,0,0]
    elif Bat_Team == 'PAK':
        temp_array += [0,0,0,0,0,1,0,0,0]
    elif Bat_Team == 'SF':
        temp_array += [0,0,0,0,0,0,1,0,0]
    elif Bat_Team == 'SL':
        temp_array += [0,0,0,0,0,0,0,1,0]
    elif Bat_Team == 'WI':
        temp_array += [0,0,0,0,0,0,0,0,1]
    
    
    if Bowl_Team == 'AUS':
        temp_array += [1,0,0,0,0,0,0,0,0]
    elif Bowl_Team == 'BAN':
        temp_array += [0,1,0,0,0,0,0,0,0]
    elif Bowl_Team == 'ENG':
        temp_array += [0,0,1,0,0,0,0,0,0]
    elif Bowl_Team == 'IND':
        temp_array += [0,0,0,1,0,0,0,0,0]
    elif Bowl_Team == 'NZ':
        temp_array += [0,0,0,0,1,0,0,0,0]
    elif Bowl_Team == 'PAK':
        temp_array += [0,0,0,0,0,1,0,0,0]
    elif Bowl_Team == 'SF':
        temp_array += [0,0,0,0,0,0,1,0,0]
    elif Bowl_Team == 'SL':
        temp_array += [0,0,0,0,0,0,0,1,0]
    elif Bowl_Team == 'WI':
        temp_array += [0,0,0,0,0,0,0,0,1]
            
    
    temp_array += [overs, total_score, total_wickets, prev_30_runs, prev_30_wickets, prev_30_dot_balls, prev_30_boundaries, prev_30_run_rate, prev_30_strike_rate]
    data = np.array([temp_array])
    my_prediction = int(LR_model.predict(data))
    
    print('Predicted Score: ',my_prediction)
    print('Predicted Score Range: ',my_prediction - meanError, 'to',my_prediction + meanError)
        

### Current Senario of the Game

In [15]:
Bat_Team = 'AUS'

Bowl_Team = 'IND'

overs = 10.3

total_score = 67

total_wickets = 2

prev_30_runs = 37

prev_30_wickets = 1

prev_30_dot_balls = 8

prev_30_boundaries = 5

prev_30_run_rate = 6*prev_30_runs/30

prev_30_strike_rate = 100*prev_30_runs/30

score_prediction(Bat_Team, Bowl_Team, overs, total_score, total_wickets, prev_30_runs, prev_30_wickets, prev_30_dot_balls, prev_30_boundaries, prev_30_run_rate, prev_30_strike_rate)

Predicted Score:  143
Predicted Score Range:  125 to 161
